<a href="https://colab.research.google.com/github/nasthosh/kaggle_competition_python/blob/data_modeling/data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import files
import numpy as np
import pandas as pd
import io
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import re
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM,Dense, SpatialDropout1D, Dropout
from keras.initializers import Constant
from keras.optimizers import Adam
from tqdm import tqdm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
upload_data = files.upload()

Saving train.csv to train.csv


In [10]:
train_dataset = pd.read_csv(io.StringIO(upload_data['train.csv'].decode('utf-8')))
train_dataset.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [ ]:
#test_dataset = pd.read_csv(io.StringIO(upload_data['test.csv'].decode('utf-8')))
#test_dataset.text.describe()

In [11]:
def remove_noise(text): 
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

sent_word_tokens = []
STOP_WORDS = nltk.corpus.stopwords.words()

for tweet in train_dataset.text:
    tweet = remove_noise(tweet)
    word_tokens = nltk.word_tokenize(tweet)
    
    #print(word_tokens)
    temp = []
    for word in word_tokens:
        #print(word)
        if word not in STOP_WORDS:
            #print("Added - " + word)
            temp.append(word)
    #print(temp)
    sent_word_tokens.append(" ".join(temp))
    #break

In [12]:
#print(temp)
sent_word_tokens[:5]

['Our Deeds Reason earthquake May ALLAH Forgive us',
 'Forest fire near La Ronge Sask Canada',
 'All residents asked shelter place notified officers No evacuation shelter place orders expected',
 'people receive wildfires evacuation orders California',
 'Just got sent photo Ruby Alaska smoke wildfires pours school']

In [13]:
train_dataset['processed_text'] = sent_word_tokens

In [14]:
print(train_dataset)

         id keyword  ... target                                     processed_text
0         1     NaN  ...      1   Our Deeds Reason earthquake May ALLAH Forgive us
1         4     NaN  ...      1              Forest fire near La Ronge Sask Canada
2         5     NaN  ...      1  All residents asked shelter place notified off...
3         6     NaN  ...      1  people receive wildfires evacuation orders Cal...
4         7     NaN  ...      1  Just got sent photo Ruby Alaska smoke wildfire...
...     ...     ...  ...    ...                                                ...
7608  10869     NaN  ...      1  Two giant cranes holding bridge collapse nearb...
7609  10870     NaN  ...      1  ariaahrary TheTawniest The control wild fires ...
7610  10871     NaN  ...      1                                   S Volcano Hawaii
7611  10872     NaN  ...      1  Police investigating ebike collided car Little...
7612  10873     NaN  ...      1  The Latest More Homes Razed Northern Californi...

[76

In [15]:
data = train_dataset[['processed_text','target']]
data

,processed_text,target
0,Our Deeds Reason earthquake May ALLAH Forgive us,1
1,Forest fire near La Ronge Sask Canada,1
2,All residents asked shelter place notified off...,1
3,people receive wildfires evacuation orders Cal...,1
4,Just got sent photo Ruby Alaska smoke wildfire...,1
...,...,...
7608,Two giant cranes holding bridge collapse nearb...,1
7609,ariaahrary TheTawniest The control wild fires ...,1
7610,S Volcano Hawaii,1
7611,Police investigating ebike collided car Little...,1


In [16]:
max_length = 20
word_length = len(data.processed_text[0].split())
padded_text_length = max_length - word_length

data.processed_text[0].rjust(padded_text_length,'0')
print(data.processed_text[0])

Our Deeds Reason earthquake May ALLAH Forgive us


In [ ]:
train_remove_stopwords.to_csv('trainClean.csv')
test_remove_stopwords.to_csv('testClean.csv')

In [ ]:
files.download('trainClean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('testClean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
train_remove_stopwords.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [ ]:
train_clean = train_remove_stopwords.drop('keyword', axis = 1)

In [ ]:
train_clean.head(5)

,id,location,text,target
0,1,NaN,"[Our, Deeds, Reason, earthquake, May, ALLAH, F...",1
1,4,NaN,"[Forest, fire, near, La, Ronge, Sask, Canada]",1
2,5,NaN,"[All, residents, asked, shelter, place, notifi...",1
3,6,NaN,"[people, receive, wildfires, evacuation, order...",1
4,7,NaN,"[Just, got, sent, photo, Ruby, Alaska, smoke, ...",1


In [ ]:
columns_to_drop = ["id","keyword","location"]
train_clean = train_remove_stopwords.drop(columns_to_drop, axis = 1)

In [ ]:
train_clean.head(5)

,text,target
0,"[Our, Deeds, Reason, earthquake, May, ALLAH, F...",1
1,"[Forest, fire, near, La, Ronge, Sask, Canada]",1
2,"[All, residents, asked, shelter, place, notifi...",1
3,"[people, receive, wildfires, evacuation, order...",1
4,"[Just, got, sent, photo, Ruby, Alaska, smoke, ...",1


In [17]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [18]:
file_path = r"/content/gdrive/My Drive/Glove/glove.6B.50d.txt"

In [19]:
embeddings_dict = {}
with open(file_path, 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [20]:
len(embeddings_dict)

400000

In [21]:
embeddings_dict['head']

array([-0.28054  ,  0.30915  , -0.20189  ,  0.029534 ,  1.1279   ,
       -0.16588  , -0.34552  , -0.1501   , -0.10558  , -1.3804   ,
        0.50196  ,  0.34911  , -1.1159   ,  0.75248  , -0.062459 ,
        0.26311  , -0.014602 ,  0.4082   , -0.40946  , -0.31068  ,
       -0.55742  ,  0.19177  ,  0.1613   , -0.52347  , -0.4502   ,
       -2.1395   ,  0.017281 ,  0.78838  , -1.0661   , -0.0098552,
        2.7098   ,  0.13188  , -0.18186  ,  0.045039 , -0.54424  ,
        0.44033  , -0.13247  ,  0.29738  ,  1.1019   ,  0.1861   ,
       -0.40799  ,  0.95422  , -0.59251  , -0.22513  ,  0.52367  ,
       -0.30058  , -0.33182  , -0.20148  ,  0.010825 , -0.15397  ],
      dtype=float32)

In [22]:
max_length = 20
word_list = data.processed_text[0].split()
word_length = len(word_list)
padded_text_length = max_length - word_length
#padded_text_length = 10
final_list = [0] * padded_text_length + word_list
final_list

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 'Our',
 'Deeds',
 'Reason',
 'earthquake',
 'May',
 'ALLAH',
 'Forgive',
 'us']

In [23]:
MAX_LEN=20
tokenizer_obj=Tokenizer()
tokenizer_obj.fit_on_texts(data.processed_text)
sequences=tokenizer_obj.texts_to_sequences(data.processed_text)
sequences[0]

[572, 4241, 769, 167, 62, 1570, 4242, 15]

In [24]:
tweet_pad=pad_sequences(sequences,maxlen=MAX_LEN,truncating='pre',padding='pre')
tweet_pad[:2]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,  572, 4241,  769,  167,   62, 1570, 4242,   15],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,  109,    7,  145,  644, 6359, 6360, 1243]],
      dtype=int32)

In [25]:
word_index=tokenizer_obj.word_index

In [26]:
num_words=len(word_index)+1
embedding_matrix=np.zeros((num_words,50))

for word,i in tqdm(word_index.items()):
    if i < num_words:
        emb_vec=embeddings_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i]=emb_vec

embedding_matrix[572]           

100%|██████████| 16759/16759 [00:00<00:00, 506607.81it/s]


array([ 0.3466    ,  0.40689   , -0.079036  , -0.57072002,  1.19770002,
       -0.51463997, -0.2388    , -0.034331  ,  0.57132   , -0.27414   ,
        0.073571  ,  0.75261998, -0.72829998,  0.071427  , -0.053052  ,
        0.40254   ,  0.46029001, -0.034256  ,  0.85873002, -0.83385998,
       -0.25567999,  0.64828002,  0.0048025 , -0.33206001,  0.64315999,
       -1.81389999, -1.15840006, -0.132     ,  1.10309994, -0.37694001,
        3.90039992,  1.43529999, -0.54368001, -0.55532998, -0.4973    ,
        0.12035   , -0.44885001, -0.10704   , -0.18783   , -0.21098   ,
       -0.46421   , -0.69106001,  0.17528   ,  0.38369   , -0.011065  ,
        0.33085001, -0.37935999,  0.31382   , -0.18248001,  0.10831   ])

In [27]:
embeddings_dict['our']

array([ 0.3466   ,  0.40689  , -0.079036 , -0.57072  ,  1.1977   ,
       -0.51464  , -0.2388   , -0.034331 ,  0.57132  , -0.27414  ,
        0.073571 ,  0.75262  , -0.7283   ,  0.071427 , -0.053052 ,
        0.40254  ,  0.46029  , -0.034256 ,  0.85873  , -0.83386  ,
       -0.25568  ,  0.64828  ,  0.0048025, -0.33206  ,  0.64316  ,
       -1.8139   , -1.1584   , -0.132    ,  1.1031   , -0.37694  ,
        3.9004   ,  1.4353   , -0.54368  , -0.55533  , -0.4973   ,
        0.12035  , -0.44885  , -0.10704  , -0.18783  , -0.21098  ,
       -0.46421  , -0.69106  ,  0.17528  ,  0.38369  , -0.011065 ,
        0.33085  , -0.37936  ,  0.31382  , -0.18248  ,  0.10831  ],
      dtype=float32)

In [28]:
tweet_pad[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,  572, 4241,  769,  167,   62, 1570, 4242,   15], dtype=int32)

In [29]:
model=Sequential()

embedding=Embedding(num_words,50,embeddings_initializer=Constant(embedding_matrix),
                   input_length=MAX_LEN,trainable=False)

model.add(embedding)
model.add(SpatialDropout1D(0.2))
model.add(LSTM(50, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))


optimzer=Adam(learning_rate=3e-4)

model.compile(loss='binary_crossentropy',optimizer=optimzer,metrics=['accuracy'])

In [30]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 50)            838000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 20, 50)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 50)                20200     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 858,251
Trainable params: 20,251
Non-trainable params: 838,000
_________________________________________________________________
